In [1]:
#Load Library
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
import string
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
import scipy as sp
import numpy as np
from scipy.linalg import svd 
from numpy import dot
from nltk.corpus import stopwords # preprocessing
from nltk.stem import PorterStemmer # preprocessing bahasa inggris
from scipy.linalg import svd
import numpy

In [2]:
kalimat = input()

#Ikatan Dokter Anak Indonesia ( IDAI) memberikan sejumlah rekomendasi untuk mencegah penularan Covid-19 kepada anak di masa pandemi. Salah satu yang direkomendasikan adalah agar anak tidak keluar rumah selama situasi Covid-19 di Indonesia belum memenuhi kriteria epidemiologi badan kesehatan dunia (WHO). Kami merekomendasikan agar anak-anak jangan keluar rumah dulu. Termasuk untuk kegiatan tatap muka di sekolah, ujar anggota Tim Satgas Penanganan Covid-19 IDAI, Yogi Prawira. Rekomendasi untuk tidak keluar rumah ini, berlaku hingga daerah tempat tinggal anak-anak dianggap sudah dapat mengatasi penularan Covid-19 lewat transmisi lokal. Namun, kata Yogi, rekomendasi ini dikecualikan jika ada keperluan mendesak yang membuat anak untuk keluar rumah

Ikatan Dokter Anak Indonesia ( IDAI) memberikan sejumlah rekomendasi untuk mencegah penularan Covid-19 kepada anak di masa pandemi. Salah satu yang direkomendasikan adalah agar anak tidak keluar rumah selama situasi Covid-19 di Indonesia belum memenuhi kriteria epidemiologi badan kesehatan dunia (WHO). "Kami merekomendasikan agar anak-anak jangan keluar rumah dulu. Termasuk untuk kegiatan tatap muka di sekolah," ujar anggota Tim Satgas Penanganan Covid-19 IDAI, Yogi Prawira. Rekomendasi untuk tidak keluar rumah ini, berlaku hingga daerah tempat tinggal anak-anak dianggap sudah dapat mengatasi penularan Covid-19 lewat transmisi lokal. Namun, kata Yogi, rekomendasi ini dikecualikan jika ada keperluan mendesak yang membuat anak untuk keluar rumah


In [3]:
# Preprocessing

stemmer = StemmerFactory().create_stemmer()  # Object stemmer
remover = StopWordRemoverFactory().create_stop_word_remover()  # objek stopword
translator = str.maketrans('', '', string.punctuation)

def stemmerEN(text):
    porter = PorterStemmer()
    ## stop = set(stopwords.words('english')) #stopwods berguna untuk
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()
    text = text.lower()
    text = [i for i in text.lower().split() if i not in stopwords]
    text = ' '.join(text)
    preprocessed_text = text.translate(translator)
    text_stem = porter.stem(preprocessed_text)
    return text_stem

def preprocessing(text):
    text = text.lower()
    text_clean = remover.remove(text)
    text_stem = stemmer.stem(text_clean)
    text_stem = stemmerEN(text_stem)
    return text_stem

def splitParagraphIntoSentences(paragraph):
    sentenceEnders = re.compile('[.!?]')
    sentenceList = sentenceEnders.split(paragraph)
    return sentenceList

pemisal_kalimat = splitParagraphIntoSentences(kalimat)
simpan_sementara_isi_berita = list()
berita_asli = list()
for per_kalimat in pemisal_kalimat:
    simpan_sementara_isi_berita.append(preprocessing(per_kalimat.strip()))
    berita_asli.append(per_kalimat.strip())
    
#tampikan hasil dari pemisahan titik dan preprocessing
print(simpan_sementara_isi_berita)

['ikat dokter anak indonesia ida beri jumlah rekomendasi cegah tular covid19 anak masa pandemi', 'salah satu rekomendasi anak keluar rumah lama situasi covid19 indonesia penuh kriteria epidemiologi badan sehat dunia who', 'rekomendasi anak jangan keluar rumah dulu', 'masuk giat tatap muka sekolah ujar anggota tim satgas tangan covid19 ida yogi prawira', 'rekomendasi keluar rumah laku hingga daerah tempat tinggal anak anggap atas tular covid19 lewat transmisi lok', 'kata yogi rekomendasi perlu desak buat anak keluar rumah']


In [4]:
# TF-IDF

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords,smooth_idf=False, norm=None)
X = tfidf_vectorizer.fit_transform(simpan_sementara_isi_berita)
return_TFIDF  = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names()).T
print(return_TFIDF)

                     0         1         2         3         4         5
anak          2.364643  1.182322  1.182322  0.000000  1.182322  1.182322
anggap        0.000000  0.000000  0.000000  0.000000  2.791759  0.000000
anggota       0.000000  0.000000  0.000000  2.791759  0.000000  0.000000
atas          0.000000  0.000000  0.000000  0.000000  2.791759  0.000000
badan         0.000000  2.791759  0.000000  0.000000  0.000000  0.000000
beri          2.791759  0.000000  0.000000  0.000000  0.000000  0.000000
buat          0.000000  0.000000  0.000000  0.000000  0.000000  2.791759
cegah         2.791759  0.000000  0.000000  0.000000  0.000000  0.000000
covid19       1.405465  1.405465  0.000000  1.405465  1.405465  0.000000
daerah        0.000000  0.000000  0.000000  0.000000  2.791759  0.000000
desak         0.000000  0.000000  0.000000  0.000000  0.000000  2.791759
dokter        2.791759  0.000000  0.000000  0.000000  0.000000  0.000000
dulu          0.000000  0.000000  2.791759  0.00000

In [5]:
#ini adalah fungsi global untuk pemanggilan Kalimat yang sudah di hasilkan berdasarkan summary CLSA.LSA
def summary_sentence(doc, angka, result_TFIDF, types=''):
    types = types.lower()
    angkas = int(angka)
    # try :
    if (types == 'lsa') :
        matrixAT=result_TFIDF.toarray()
        matrixA=np.transpose(matrixAT)
        matrixAT=np.transpose(matrixA)
        a = np.array(matrixAT)
        b = np.array(matrixA)
        aTa = dot(a,b)
        tempSVD = np.array(aTa)
        u, s, v = svd(tempSVD)
        s = sp.diag(s)
        datas = list()
        loop = v[0]
        if(angkas <= len(s[0])):
            for x in range(0, 8):
                print(x)
                loop = s[0]
                temp2 = 0
                temp = list()
                for i in np.arange(np.size(loop)):
                    for j in np.arange(np.size(loop)):
                        temp2 = temp2 + np.sqrt(dot(np.square(s[i][j]),np.square(v[i][j])))
                    if(temp2 == temp[x]):
                        if(temp2 != 0):
                            print(doc[i])
                    temp2 = 0
        else:
            print('Jumlah input melebihi jumlah dokumen')
        return datas
            
    elif (types == 'clsa') :
        matrixAT=result_TFIDF.toarray()
        matrixA=np.transpose(matrixAT)
        matrixAT=np.transpose(matrixA)
        a = np.array(matrixAT)
        b = np.array(matrixA)
        aTa = dot(a,b)
        tempSVD = np.array(aTa)
        u, s, v = svd(tempSVD)
        s = sp.diag(s)
        aa =v[0]
        for i in np.arange(np.size(aa)):
            av = np.average(v[i])
            for j in np.arange(np.size(aa)):
                if v[i][j] < av :
                    v[i][j] = 0

        loop = s[0]
        temp = 0
        data = list()
        for i in np.arange(np.size(loop)):
            for j in np.arange(np.size(loop)):
                temp = temp + np.sqrt(dot(np.square(s[i][j]),np.square(v[i][j])))
            if(temp != 0):
                data.append(doc[i])
            temp = 0
        return data

In [6]:
#kemudian kita cari CLSA dari hasil TF-IDF

def CLSA(tfidf):
    matrixAT=tfidf.toarray()
    matrixA=numpy.transpose(matrixAT)
    matrixAT=numpy.transpose(matrixA)
    a = np.array(matrixAT)
    b = np.array(matrixA)
    aTa = dot(a,b)
    A = np.array(aTa)
    u, s, v = np.linalg.svd(A, full_matrices = False)
    s = sp.diag(s)
    aa =v[0]
    for i in np.arange(np.size(aa)):
        av = np.average(v[i])
        for j in np.arange(np.size(aa)):
            if v[i][j] < av :
                v[i][j] = 0
    loop = s[0]
    for i in np.arange(np.size(loop)):
        for j in np.arange(np.size(loop)):
            s[i][j] = np.sqrt(dot(np.square(s[i][j]),np.square(v[i][j])))
    return s

def sum_frame_by_column(frame, new_col_name, list_of_cols_to_sum):
    frame[new_col_name] = frame[list_of_cols_to_sum].astype(float).sum(axis=1)
    return(frame)

a = int(input())
return_CLSA = pd.DataFrame(CLSA(X)).T
sentences_clsa = summary_sentence(berita_asli,a, X, types='clsa')
rank_CLSA = sum_frame_by_column(return_CLSA, 'total_score_document', [i for i in range(len(return_CLSA[0]))])
soritng = sorted(rank_CLSA['total_score_document'], reverse=True)
# print(return_CLSA)
print(sentences_clsa)
# print(rank_CLSA)
# print(soritng)

2
['"Kami merekomendasikan agar anak-anak jangan keluar rumah dulu', 'Rekomendasi untuk tidak keluar rumah ini, berlaku hingga daerah tempat tinggal anak-anak dianggap sudah dapat mengatasi penularan Covid-19 lewat transmisi lokal']


In [7]:
#kemudian kita banding kan dengan metode LSA
def LSA(tfidf): 
    matrixAT=tfidf.toarray()
    matrixA=np.transpose(matrixAT)
    matrixAT=np.transpose(matrixA)
    a = np.array(matrixAT)
    b = np.array(matrixA)
    aTa = dot(a,b)
    tempSVD = np.array(aTa)
    u, s, v = np.linalg.svd(tempSVD, full_matrices = False)
    s = sp.diag(s)
    loop = s[0]
    for i in np.arange(np.size(loop)):
        for j in np.arange(np.size(loop)):
            s[i][j] = np.sqrt(dot(np.square(s[i][j]),np.square(v[i][j])))
    return s

a = int(input())
return_LSA = pd.DataFrame(LSA(X)).T
sentences_lsa = summary_sentence(berita_asli,a, X, types='lsa')

print(return_LSA)
print(sentences_lsa)


2
0


IndexError: list index out of range